# Codes for manul process in HeuDiConv step
    1. Test whether new sequence descripions apreas in the coming data;
    2. Codes for livingPark: Obtaining information from .xml meta data and test whether new T1 SERIES descriptions appear and also validate the subject session labels for additional livingPark subjects from metadata accompanied the images (.xml file for each image).


In [1]:
# from pathlib import Path
import sys
from pathlib import Path
import glob
import os
import shutil
import numpy as np
import pandas as pd

# system PATH
code_path_str='/scratch/mr_proc/'
sys.path.append(code_path_str)

# study PATH
code_dir = Path(code_path_str)
xml_path = Path('/scratch/0tmp/meta_data') # The meta data folder on local machine.
xml_out_path = Path(code_dir / 'metadata' / 'ppmi' / 'PPMI_livingpark2_metadata.csv')

In [15]:
# read meta data
import xml.etree.ElementTree as ET

files = [f for f in os.listdir(xml_path) if f[-3:]=='xml']
tree = ET.parse(str(xml_path/files[0]))
root = tree.getroot()
subject_info_list= [];
for file in files:
    tree = ET.parse(str(xml_path/file))
    root = tree.getroot()
    data_dict={};
    data_dict['Image Data ID']=root[0][3][4][6][0].text
    data_dict['Subject']=root[0][3][0].text
    data_dict['Group']=root[0][3][1].text
    data_dict['Sex']=root[0][3][2].text
    data_dict['Visit']=root[0][3][3][0].text
    data_dict['Age']=root[0][3][4][1].text
    data_dict['Modality']=root[0][3][4][5][1].text
    data_dict['Description']=root[0][3][4][6][1].text
    data_dict['Acq Date']=root[0][3][4][5][2].text
    subject_info_list.append(data_dict)
df=pd.DataFrame(subject_info_list)
df

,Image Data ID,Subject,Group,Sex,Visit,Age,Modality,Description,Acq Date
0,1537582,110955,Prodromal,M,Baseline,68.56,MRI,3D T2 SAG FLAIR_ND,2022-01-18
1,1537581,110955,Prodromal,M,Baseline,68.56,MRI,3D T2 SAG FLAIR,2022-01-18
2,1537571,110955,Prodromal,M,Baseline,68.56,MRI,AX T2 GRE MT,2022-01-18
3,1537572,110955,Prodromal,M,Baseline,68.56,MRI,AX T2 GRE MT,2022-01-18
4,1537573,110955,Prodromal,M,Baseline,68.56,MRI,AX T2 GRE MT,2022-01-18
...,...,...,...,...,...,...,...,...,...
591,1348316,75423,Prodromal,M,Baseline,72.08,MRI,Cal Head 24,2018-10-25
592,1199314,75447,Prodromal,M,Baseline,63.59,MRI,AXIAL PD-T2 TSE FS,2018-12-10
593,1199315,75447,Prodromal,M,Baseline,63.59,MRI,AXIAL PD-T2 TSE FS,2018-12-10
594,1199316,75447,Prodromal,M,Baseline,63.59,MRI,MPRAGE_GRAPPA_ADNI,2018-12-10


In [2]:
# T1 sequence description,copied from heuristics.
T1W_SERIES = [
    'MPRAGE 2 ADNI',
    'MPRAGE ADNI',
    'MPRAGE GRAPPA 2',
    'MPRAGE GRAPPA2',
    'MPRAGE GRAPPA2(adni)',
    'MPRAGE w/ GRAPPA',
    'MPRAGE_GRAPPA',
    'MPRAGE_GRAPPA_ADNI',
    'MPRAGE GRAPPA',
    'SAG T1 3D MPRAGE',
    'sag mprage',
    'MPRAGEadni',
    'MPRAGE GRAPPA_ND',
    '3D SAG',
    'MPRAGE T1 SAG',
    'MPRAGE SAG',
    'SAG T1 3DMPRAGE',
    'SAG T1 MPRAGE',
    'SAG 3D T1',
    'SAG MPRAGE GRAPPA2-NEW2016',
    'SAG MPRAGE GRAPPA_ND',
    'Sag MPRAGE GRAPPA',
    'AXIAL T1 3D MPRAGE',
    'SAG MPRAGE GRAPPA',
    'sT1W_3D_FFE',
    'sT1W_3D_ISO',
    'sT1W_3D_TFE',
    'sag 3D FSPGR BRAVO straight',
    'SAG T1 3D FSPGR',
    'SAG FSPGR 3D '
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG T1 3D FSPGR 3RD REPEAT',
    'SAG FSPGR BRAVO',
    'SAG SPGR 3D',
    'SAG 3D SPGR',
    'FSPGR 3D SAG',
    'SAG FSPGR 3D',
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG FSPGR 3D ',
    't1_mpr_ns_sag_p2_iso',
    'T1',
    'T1 Repeat',
    'AX T1',
    'axial spgr',
    'T1W_3D_FFE AX',
    # added by Vincent
    'AX T1 SE C+',
    '3D SAG T1 MPRAGE',
    '3D SAG T1 MPRAGE_ND',
    '3D T1',
    '3D T1 MPRAGE',
    '3D T1-weighted',
    'Accelerated Sag IR-FSPGR',
    'MPRAGE',
    'MPRAGE - Sag',
    'MPRAGE Phantom GRAPPA2',
    'MPRAGE w/ GRAPPA 2',
    'PPMI_MPRAGE_GRAPPA2',
    'SAG 3D T1 FSPGR',
    'SAG FSPGR 3D VOLUMETRIC T1',
    'Sag MPRAGE GRAPPA_ND',
    'T1-weighted, 3D VOLUMETRIC',
    'tra_T1_MPRAGE', 
    '3D T1-weighted_ND', ## added from livingpark
    '3D T1 _weighted',
    'Sagittal 3D Accelerated MPRAGE',
    'T1 REPEAT',
    'MPRAGE Repeat',
    'SAG_3D_MPRAGE',
    'T1-weighted,_3D_VOLUMETRIC', ## added from all T1 PPMI
    '3D_SAG',
    'FSPGR_3D_SAG',
    'SAG_T1_MPRAGE',
    'MPRAGE_2_ADNI',
    'MPRAGE_Repeat',
    'T1W_3D_FFE_COR',
    'SAG',
    'SAG_MPRAGE_GRAPPA2-NEW2016',
    'MPRAGE_GRAPPA_2',
    'AX_T1_SE_C+',
    '3D_Sagittal_T1',
    'Coronal',
    'MPRAGE_SENSE2',
    'sag_mprage',
    'Accelerated_Sag_IR-FSPGR',
    'SAG_SPGR',
    'MPRAGE_GRAPPA2_adni_',
    'Sag_MPRAGE_GRAPPA',
    '3D_SAG_T1_MPRAGE',
    'MPRAGE_ADNI',
    'AX_3D_FSPGR_straight_brain_lab',
    '3D_T1_MPRAGE',
    'rpt_PPMI_MPRAGE_GRAPPA2',
    'SAG_FSPGR_BRAVO',
    '3D_T1',
    'sag_3D_FSPGR_BRAVO_straight',
    'mprage',
    'AX_T1',
    'Sagittal_3D_Accelerated_MPRAGE',
    'MPRAGE_w__GRAPPA',
    'SAG_T1_3D_FSPGR',
    'SAG_3D_MPRAGE_RPT',
    'SAG_T1_SE',
    'axial_spgr',
    'SAG_T1_3D_FSPGR_3RD_REPEAT',
    'SAG_FSPGR_3D',
    'MPRAGE_SAG',
    'T1_repeat',
    'T1_SAG',
    'Sag_T1',
    'T1_REPEAT',
    'T1W_3D_FFE_AX',
    '3D_T1-weighted',
    'SAG_T1_3DMPRAGE',
    'MPRAGE_T1_SAG',
    'SAG_FSPGR_3D_VOLUMETRIC_T1',
    'SAG_3D_T1_FSPGR',
    'SAG_3D_T1',
    'T1_Repeat',
    'SAG_SPGR_3D',
    'MPRAGE_GRAPPA2', 
    'MPRAGE_-_Sag',
    'SAG_3D_FSPGR_BRAVO_STRAIGHT',
    'SAG_MPRAGE_GRAPPA'
]

# converted
T2W_SERIES = [
    # single echo only
    't2_tse_tra',
    't2 cor',
    'T2 COR',
    'T2W_TSE',
    'AX T2',
    'AX T2 AC-PC LINE ENTIRE BRAIN',
    'AX T2 AC-PC line Entire Brain',
    'Ax T2 Fse thin ac-pc',
    # mixed single / dual-echo
    'AXIAL FSE T2 FS',
    'T2',
    '*AX FSE T2', # added from livingpark
    'AXIAL  T2  FSE',
    '_AX_FSE_T2',
    ' 3-pl_T2__FGRE'
]

# converted as T2 with acq-GREMT, according to study design
MT_SERIES = [
    ## T2 MT added by Vincent 
    '2D GRE - MT',
    '2D GRE MT',
    '2D GRE-MT',
    '2D GRE-MT_RPT2',
    '2D GRE-NM',
    '2D GRE-NM_MT',
    '2D_GRE-MT',
    'AX GRE -MT',
    'AXIAL 2D GRE-MT',
    'LOWER 2D GRE MT',
    '2D GRE MT MTC-NO', # added from living park
    'NM-MT', # added from all T1 PPMI
    '2D_GRE-NM',
    '2D_GRE_-_MT',
    'SAG_3D_SPGR',
    'MPRAGE_w__GRAPPA_2',
    'T1_AXIAL',
    'sag',
    'SAG_T1_3D_MPRAGE',
    'AXIAL_T1_3D_MPRAGE',
    '2D_GRE_MT',
    'AXIAL_2D_GRE-MT',
    '2DGRE-MT',
    '2D_GRE-MT_RPT2', 
    'AX_GRE_-MT',
    '2D_GRE_MT_MTC-NO',
    'AX_2D_GRE-MT',
    '2D_GRE-NM_MT',
    'AX_T2_GRE_MT',
    'LOWER_2D_GRE_MT',
]

# converted
T2_STAR_SERIES = [
    'AXIAL_T2_STAR'
]

# converted
PDT2_SERIES = [
    'AX DE TSE',
    'AX DUAL_TSE',
    'DUAL_TSE',
    'sT2W/PD_TSE',
    'Axial PD-T2-FS TSE',
    'Axial PD-T2 TSE',
    'Axial PD-T2 TSE FS',
    'AXIAL PD-T2 TSE FS',
    'AX PD + T2',
    'PD-T2 DUAL AXIAL TSE',
    'Axial PD-T2 TSE_AC/PC line',
    'Axial PD-T2 TSE_AC PC line',
    'Ax PD /T2',
    'AXIAL PD+T2 TSE',
    'AX T2 DE',
    't2 weighted double echo',
    'Axial_PD_T2', ## Added from all T1 PPMI version
    'AX_T2',
    'Axial_PD-T2_TSE_FS',
    'AX_PD_+_T2',
    'Ax_PD__T2',
    'Axial_PD-T2_TSE',
    'AX_DUAL_TSE',
]

# converted
FLAIR_SERIES = [
    # FLAIR (no weighting specified)
    'FLAIR_LongTR AX',
    'FLAIR_LongTR SENSE',
    'AX FLAIR',
    'AXIAL FLAIR',
    'FLAIR_longTR',
    'FLAIR AXIAL',
    'ax flair',
    'Cor FLAIR TI_2800ms',
    'FLAIR',
    # T2 FLAIR
    'AX T2 FLAIR',
    'T2  AXIAL FLAIR',
    'Ax T2 FLAIR  ang to ac-pc',
    'T2W_FLAIR',
    'AX FLAIR T2',
    'AX T2 FLAIR 5/1',
    'Ax T2 FLAIR',
    't2_tirm_tra_dark-fluid_',
    't2_tirm_tra_dark-fluid NO BLADE',
    # T1 FLAIR -- should these be here?
    'Ax T1 FLAIR',
    # added by Vincent
    '3D T2  SPC FLAIR C9C1HN007',
    '3D T2 FLAIR',
    '3D T2 FLAIR_ND',
    '3D T2 FLAIR_SAGITAL',
    '3D T2 FLAIR_ti1650',
    '3D T2 FLAIR_ti1650_ND',
    '3D_Brain_VIEW_FLAIR_SAG',
    '3D_T2_FLAIR',
    '3D_T2_FLAIR_SAG INVICCRO T2 FLAIR',
    'SAG 3D FLAIR',
    'SAG CUBE FLAIR',
    'Sag 3D T2 FLAIR',
    'SAG 3D T2 FLAIR_',  # added from livingpark
    'AX_T2_FLAIR', # added from all T1
    'Sag_MPRAGE_GRAPPA_ND',
    'FLAIR_LongTR_AX',
    'MPRAGE_GRAPPA_ND',
    'Ax_T2_FLAIR',
    'SAG_MPRAGE_GRAPPA_ND',
    'AX_FLAIR',
    '3D_SAG_T1_MPRAGE_ND',
]



# Codes to search for new sequence descriptions in PPMI

In [3]:
# dicom info from PPMI_all_T1 version of dataset
new_dicom_info_file = code_dir / 'metadata' / 'ppmi' / 'PPMI_all_T1_7dcminfo.csv'
dicom_df = pd.read_csv(new_dicom_info_file)
" <-> ".join(list(set(dicom_df.Modality.unique()) - set(T1W_SERIES) - set(T2W_SERIES) - set(MT_SERIES) - 
set(T2_STAR_SERIES) - set(PDT2_SERIES) - set(FLAIR_SERIES)))

'3_PLANE_LOC <-> MPRAGE_ASO <-> MPRAGE_ASO_repeat <-> STRUC_BRAVO_SAG3D_ARC <-> Transverse <-> COR <-> TRA <-> B0rf_Map <-> SURVEY <-> Cal_Head_24 <-> 3_plane <-> MIDLINE_SAG_LOC <-> localizer <-> MPRAGE_64channel_p2 <-> GRE_B0 <-> MPRAGE_Phantom_GRAPPA2 <-> MR <-> 3_Plane_Localizer <-> 3-pl_T2__FGRE <-> DTI_gated'

# Codes for new livingPark new subjects

In [17]:
# test wheter there are new T1 SERIES in coming data and save the merged meta data csv file.
print("Mannully check whether there is new T1 sequences:")
display(set(df.Description.unique())-set(T1W_SERIES))

# Logs: 
# for livingpark1, new serise has added;
# for livingpark2, no new series added;

df=pd.DataFrame(subject_info_list)
def encode_visit(x_):
    if x_=='Baseline':
        return 1 # 
    elif x_=='Screening':
        return 0
    else:
        return -1
df.loc[:,'Visit'] = [ encode_visit(x) for x in df['Visit'] ]
df_baseline=df[df['Visit']>=0].copy()
#df["T1 include"] = [1 if x in T1W_SERIES  else 0 for x in df['Description']]
#t1_df=df[df["T1 include"]==1].copy()
#t1_df=t1_df.drop(columns=['T1 include'])
df_baseline.loc[:,"Image Data ID"] = ["I"+str(x) for x in df_baseline['Image Data ID']]
#df=df.apply(lambda x: "\""+str(x)+"\"")
print('Final size: ', df.shape)
# save file
#xml_out_path = Path(code_dir / 'metadata'  / 'PPMI_livingpark2_metadata.csv')
df_baseline.to_csv(xml_out_path, index=False)

Mannully check whether there is new T1 sequences:


{'*AX FSE T2',
 '2D GRE MT MTC-NO',
 '2D GRE-MT',
 '3D T2 FLAIR',
 '3D T2 SAG FLAIR',
 '3D T2 SAG FLAIR_ND',
 'AX DE TSE',
 'AX DUAL_TSE',
 'AX FLAIR',
 'AX FLAIR T2',
 'AX GRE -MT',
 'AX PD + T2',
 'AX T2',
 'AX T2 DE',
 'AX T2 FLAIR',
 'AX T2 GRE MT',
 'AXIAL  T2  FSE',
 'AXIAL FLAIR',
 'AXIAL PD-T2 TSE FS',
 'Ax PD /T2',
 'Ax T2 FLAIR',
 'Ax T2 FSE with Fat Sat',
 'Axial FLAIR',
 'Axial PD-T2 TSE',
 'Axial PD-T2 TSE FS',
 'Axial PD-T2-FS TSE',
 'Axial T2 TSE with Fat Sat',
 'Axial T2-FLAIR',
 'BOLD_RS ACPC_LINE',
 'Cal Head 24',
 'Cor FLAIR TI_2800ms',
 'DTI_LR',
 'DTI_RL',
 'FLAIR',
 'FLAIR AXIAL',
 'FLAIR_LongTR AX',
 'L>R RESTING STATE FMRI ep2d_fid_basic_bold',
 'PD-T2 DUAL AXIAL TSE',
 'PPMI_AX_T2_FLAIR',
 'PPMI_Axial T2 TSE with Fat Sat',
 'R>L RESTING STATE FMRI ep2d_fid_basic_bold',
 'SAG 3D FLAIR',
 'SAG 3D MPRAGE',
 'SAG 3D T2 FLAIR_',
 'T2',
 'repeat_BOLD_RS ACPC_LINE',
 'rsfMRI_LR',
 'rsfMRI_RL'}